In [1]:
import re
import qiskit
import random
import math

In [12]:
def QASMDict(gates):
#     print(gates)
    qasmDict = {} #  {'qbit number' : []}
#     print(gates)
    count = []
    for x in range(len(gates)):
        if x == 1 or x == 0:
            continue
        if gates[x][0] == 'qreg':
            for y in range(int(gates[x][1][2])):
                qasmDict[str(y)] = []
#         print(list(qasmDict.keys()))
        if len(count) == 0:
            for b in list(qasmDict.keys()):
                count.append(0)
#         print(count)
        if gates[x][0] != 'OPENQASM' and gates[x][0] != 'include' and gates[x][0] != 'qreg' and gates[x][0] != 'creg' and gates[x][0] != 'measure':
            if len(gates[x])-1 > 1:
                if count[int(gates[x][1][2])] > count[int(gates[x][2][2])]:
                    qasmDict[str(gates[x][2][2])].append([None])
                    count[int(gates[x][2][2])] += 1
                elif count[int(gates[x][1][2])] < count[int(gates[x][2][2])]:
                    qasmDict[str(gates[x][1][2])].append([None])
                    count[int(gates[x][1][2])] += 1
                qasmDict[str(gates[x][2][2])].append(gates[x])
                qasmDict[str(gates[x][1][2])].append(gates[x])
            else:
                qasmDict[str(gates[x][1][2])].append(gates[x])
                count[int(gates[x][1][2])] += 1
#             for z in range(len(gates[x])-1):
#                 qasmDict[str(gates[x][z+1][2])].append(gates[x])
#                 count[int(gates[x][z+1][2])] += 1
#         print(count.index(min(count)))
#         print(count.index(max(count)))
        uneven = True
        while uneven:
            uneven = False
            minInd = count.index(min(count))
            maxInd = count.index(max(count))
            if count[maxInd] > count[minInd] + 1:
                qasmDict[str(minInd)].append([None])
                count[minInd] += 1
                uneven = True
#         for a in count:
            
    return qasmDict

def openQASM(qasm_loc):
    qasm_f = open(qasm_loc, "r")
    gates = []
    for line in qasm_f:
        gates.append( list(filter(None, re.split(" |,", line.strip()[:-1]))) )
    print("These are the raw gates")
#     print(gates)
    qasmDict = QASMDict(gates)
    qasm_f.close()
    return qasmDict


In [4]:
#MAIN SWAPS
def HHtoNone(qubit, depth, circuitDict):
    index2 = depth + 1
    if circuitDict[qubit][depth + 1] == []:
        for x in range(depth+1,len(circuitDict[qubit])):
            if circuitDict[qubit][x] != []:
                index2 = x
                break
    circuitDict[qubit][index2] = []
    circuitDict[qubit][depth] = []
    return circuitDict


def NonetoHH(qubit, depth, circuitDict):
    qString = 'q[' + qubit + ']'
    circuitDict[qubit].insert(depth, ['h', qString])
    circuitDict[qubit].insert(depth, ['h', qString])
    return circuitDict


# Depth is cz depth
def HZHtoCX(qubit, depth, circuitDict):
    qString = 'q[' + qubit + ']'
    circuitDict[circuitDict[qubit][depth][1][2]][depth][0] = 'cx'
    circuitDict[qubit][depth-1] = [None]
    circuitDict[qubit][depth][0] = 'cx'
    circuitDict[qubit][depth+1] = [None]
#     print(circuitDict[circuitDict[qubit][depth][1][2]][depth])
    return circuitDict

# Depth is cx depth
def CXtoHZH(qubit, depth, circuitDict):
    qString = 'q[' + qubit + ']'
    circuitDict[qubit][depth][0] = 'cz'
    circuitDict[circuitDict[qubit][depth][1][2]][depth][0] = 'cz'
    circuitDict[qubit].insert(depth+1, ['h', qString])
    circuitDict[qubit].insert(depth, ['h', qString])
    return circuitDict
    
    

In [5]:
def dictToQasm(circuitDict):
    outText = "OPENQASM 2.0;\ninclude \"qelib1.inc\";"
#     print(len(circuitDict.keys()))
    outText += '\nqreg q[' + str(len(circuitDict.keys())) + '];\ncreg c[' + str(len(circuitDict.keys())) + '];\n'
    
    maxind = 0
    for x in range(len(circuitDict.keys())):
#         print(x)
        if len(circuitDict[str(x)]) > len(circuitDict[str(maxind)]):
            maxind = x
    ind = 0
    while len(circuitDict[str(maxind)]) > ind:
        for y in range(len(circuitDict.keys())):
            if len(circuitDict[str(y)]) > ind:
                if circuitDict[str(y)][ind] == []:
                    continue
                elif len(circuitDict[str(y)][ind]) == 2:
                    outText += circuitDict[str(y)][ind][0] + ' ' + circuitDict[str(y)][ind][1] + ';\n'
                elif len(circuitDict[str(y)][ind]) == 3:
                    if circuitDict[str(y)][ind][2][2] == str(y):
                        outText += circuitDict[str(y)][ind][0] + ' ' + circuitDict[str(y)][ind][1] + ',' + circuitDict[str(y)][ind][2] + ';\n'
        ind += 1
    for z in range(len(circuitDict.keys())):
        outText += 'measure q[' + str(z) + '] -> c[' + str(z) + '];\n'
            
    return outText

In [6]:
def swapList(circuitDict):

    swapDict = {'HHtoNone': [], 'HZHtoCX': [], 'CXtoHZH': [], 'NonetoHH': []}

    for qubit in circuitDict:
        gates = circuitDict[qubit]

        # go through all the gates for a qubit
        for i in range(len(gates) - 1):
            # HHtoNone
            if (gates[i][0] == 'h') and (gates[i+1][0] == 'h'):
                swapDict['HHtoNone'].append([qubit, i])

            # HZHtoCX
            if i > 0:
                if (gates[i-1][0] == 'h') and (gates[i][0] == 'cz') and (gates[i+1][0] == 'h') and (gates[i-1][-1] == gates[i][-1]):
                    swapDict['HZHtoCX'].append([qubit, i])

            # CXtoHZH
            if (gates[i][0] == 'cx') :
                swapDict['CXtoHZH'].append([qubit, i])

            # None to HH
            swapDict['NonetoHH'].append([qubit, i])
        # None to HH
        swapDict['NonetoHH'].append([qubit, len(gates)-1])
        
    return swapDict

In [36]:
def slicer(circuitQasm, depth):
    circuit = openQASM(circuitQasm)
    base = dict(circuit)
    for qubit in base:
        base[qubit] = []
    print(circuit)
    print(base)
    leastLen = 9999999999
    maxLen = 0
    for x in circuit:
        if len(circuit[x]) < leastLen:
            leastLen = len(circuit[x])
        if len(circuit[x]) > maxLen:
            maxLen = len(circuit[x])
    if depth > leastLen or (depth == leastLen and depth == maxLen):
        return dictToQasm(circuitQasm)
    print(leastLen)
    numSubCircuits = math.floor(leastLen/depth)
    subCircuitList = []
    for subCirc in range(numSubCircuits):
        subCircuitList.append(dict(base))
        for q in subCircuitList[subCirc]:
            subCircuitList[subCirc][q] = circuit[q][0:depth]
#             print(subCircuitList)
            del circuit[q][0:depth]
    if circuit != dict(base):
        subCircuitList.append(circuit)
    qasmList = []
    print(subCircuitList)
    for dictCircuit in subCircuitList:
        qasmList.append(dictToQasm(dictCircuit))
    return qasmList
sliced = slicer('circuit.qasm', 2)
# print(sliced)
circ_out = qiskit.QuantumCircuit.from_qasm_str(sliced[3])
circ_out.draw()

    

These are the raw gates
{'0': [['cx', 'q[1]', 'q[0]'], [None], [None], ['cx', 'q[1]', 'q[0]'], ['h', 'q[0]'], ['h', 'q[0]'], ['h', 'q[0]']], '1': [['cx', 'q[1]', 'q[0]'], ['h', 'q[1]'], ['h', 'q[1]'], ['cx', 'q[1]', 'q[0]'], [None], [None]]}
{'0': [], '1': []}
6
[{'0': [['cx', 'q[1]', 'q[0]'], [None]], '1': [['cx', 'q[1]', 'q[0]'], ['h', 'q[1]']]}, {'0': [[None], ['cx', 'q[1]', 'q[0]']], '1': [['h', 'q[1]'], ['cx', 'q[1]', 'q[0]']]}, {'0': [['h', 'q[0]'], ['h', 'q[0]']], '1': [[None], [None]]}, {'0': [['h', 'q[0]']], '1': []}]


┌───┐┌─┐
q_0: ┤ H ├┤M├
     └┬─┬┘└╥┘
q_1: ─┤M├──╫─
      └╥┘  ║ 
c: 2/══╩═══╩═
       1   0

In [20]:
circuit = openQASM('circuit.qasm')
print(circuit)

swapDict = swapList(circuit)
print(swapDict)
# qubit, depth = random.choice(swapDict['HZHtoCX'])

print(HHtoNone('1', 1, circuit))

# print(CXtoHZH(qubit, depth, circuit))



These are the raw gates
[['OPENQASM', '2.0'], ['include', '"qelib1.inc"'], ['qreg', 'q[2]'], ['creg', 'c[2]'], ['cx', 'q[1]', 'q[0]'], ['h', 'q[1]'], ['h', 'q[1]'], ['cx', 'q[1]', 'q[0]'], ['measure', 'q[0]', '->', 'c[0]'], ['measure', 'q[1]', '->', 'c[1]']]
{'0': [['cx', 'q[1]', 'q[0]'], [None], [None], ['cx', 'q[1]', 'q[0]']], '1': [['cx', 'q[1]', 'q[0]'], ['h', 'q[1]'], ['h', 'q[1]'], ['cx', 'q[1]', 'q[0]']]}
{'HHtoNone': [['1', 1]], 'HZHtoCX': [], 'CXtoHZH': [['0', 0], ['1', 0]], 'NonetoHH': [['0', 0], ['0', 1], ['0', 2], ['0', 3], ['1', 0], ['1', 1], ['1', 2], ['1', 3]]}
{'0': [['cx', 'q[1]', 'q[0]'], [None], [None], ['cx', 'q[1]', 'q[0]']], '1': [['cx', 'q[1]', 'q[0]'], [], [], ['cx', 'q[1]', 'q[0]']]}


In [22]:
circ = qiskit.QuantumCircuit.from_qasm_file('circuit.qasm')
circ.draw()

┌───┐          ┌───┐┌───┐┌───┐┌───┐┌─┐
q_0: ┤ X ├──────────┤ X ├┤ H ├┤ H ├┤ H ├┤M├
     └─┬─┘┌───┐┌───┐└─┬─┘└┬─┬┘└───┘└───┘└╥┘
q_1: ──■──┤ H ├┤ H ├──■───┤M├────────────╫─
          └───┘└───┘      └╥┘            ║ 
c: 2/══════════════════════╩═════════════╩═
                           1             0

In [22]:
circ_out = qiskit.QuantumCircuit.from_qasm_str(dictToQasm(circuit))
circ_out.draw()

┌───┐┌───┐┌─┐   
q_0: ┤ X ├┤ X ├┤M├───
     └─┬─┘└─┬─┘└╥┘┌─┐
q_1: ──■────■───╫─┤M├
                ║ └╥┘
c: 2/═══════════╩══╩═
                0  1